In [ ]:
import pygui as pgui
pgui.setup_interface()
#pgui.username_widget.value = 'strudel'
#change = {'name': 'value', 'old': '', 'new': 'tmind'}
#pgui.reload_data_on_click(None,'Load Decks/Fusions')
#pgui.reload_data_on_click(None,'Create all Fusions')
#pgui.reload_data_on_click(None,'Generate Dataframe')
pgui.db_list.value = 'strudel'
#pgui.handle_db_list_change(change)

#GridManager.get_forgeborn_abilities()

event = {'name': 'row_added', 'index': 0, 'source': 'gui'}
pgui.grid_manager.filterGridObject.grid_filter_on_row_added(event, pgui.grid_manager.filterGridObject.qgrid_filter)
#pgui.reload_data_on_click(None,'Decks')
if False:
    #pgui.db_list.value = 'tmind'
    #event = {'name': 'selection_changed', 'old': [], 'new': [1], 'source': 'gui'}

    event = {'name': 'cell edited', 'index': 0, 'column': 'Active', 'new': False, 'source': 'filter'}
    pgui.grid_manager.filterGridObject.grid_filter_on_cell_edit(event,pgui.grid_manager.filterGridObject.qgrid_filter)
    #collection_grid = pgui.qm.grids.get('collection')
    #df = pgui.generate_deck_content_dataframe(event, collection_grid.main_widget)
    #df = pgui.generate_deck_statistics_dataframe()

In [ ]:
import pygui as pgui
import pandas as pd


def add_row_in_qgrid(GridManager, id=0, row_data=None, grid_manager=None):
    """
    Programmatically adds a row to the qgrid widget and triggers the row_added event.
    """
    # Ensure the deck content grid has data
    grid_id = f"filtered_grid_{id}"
    filter_df = GridManager.filterGridObject.get_changed_df()

    if filter_df is not None:
        # Add a new row to the DataFrame
        new_row = pd.DataFrame({
            'Type': ['Deck'],
            'Name': [''],
            'Modifier': [''],
            'Creature': [''],
            'Spell': ['Preserved Habitat'],
            'Forgeborn Ability': [''],
            'Active': [True],
            'Mandatory Fields': ['Name, Forgeborn Ability']
        })
        filter_df = pd.concat([filter_df, new_row], ignore_index=True)

        # Update the grid with the new DataFrame
        GridManager.filterGridObject.df = filter_df

        # Trigger the row_added event manually
        event = {
            'index': len(filter_df) - 1  # Index of the newly added row
        }

        # Call the handler if grid_manager is provided
        if GridManager:
            GridManager.filterGridObject.grid_filter_on_row_added(event, GridManager.filterGridObject.qgrid_filter)
    else:
        print("No data in the grid to add a row.")

def test_update_filter_grid(grid_manager):
    """
    Simulates updating a value in the filter grid and triggers the necessary updates.

    Args:
        grid_manager (DynamicGridManager): The grid manager instance to interact with.
    """
    # Access the filter grid
    filter_grid_widget = grid_manager.filterGridObject.qgrid_filter

    # Retrieve the current DataFrame from the filter grid
    filter_df = filter_grid_widget.get_changed_df()

    # Ensure there is data in the filter grid
    if filter_df is not None and not filter_df.empty:
        # Select the first row and update a value programmatically
        row_index = 0
        new_value = "The People of Bearing"

        # Update the 'Name' column in the first row
        filter_df.at[row_index, 'Name'] = new_value

        # Apply the updated DataFrame back to the filter grid
        filter_grid_widget.df = filter_df

        # Trigger the `cell_edited` event manually
        event = {
            'index': row_index,
            'column': 'Name',
            'old': "",  # Original value
            'new': new_value  # New value
        }
        # Trigger the cell edit handler to reflect the change in the system
        grid_manager.filterGridObject.grid_filter_on_cell_edit(event, filter_grid_widget)

        print(f"Test: Updated filter grid row {row_index} 'Name' to '{new_value}'.")
    else:
        print("Filter grid is empty or unavailable.")



def select_row_in_qgrid(GridManager, id=0, row_index=0, grid_manager=None):
    """
    Programmatically selects a row in the qgrid widget and triggers the selection event.
    """
    # Ensure the deck content grid has data
    grid_id = f"filtered_grid_{id}"
    grid_df = GridManager.get_grid_df(grid_id)

    if grid_df is not None and not grid_df.empty:
        # Check if the specified row index is valid
        if 0 <= row_index < len(grid_df):
            # Programmatically select the row
            # Get the widget first
            widget = GridManager.grids[grid_id].main_widget
            widget.change_selection([row_index])

            # Trigger the selection event manually
            event = {
                'new': [row_index]  # 'new' key contains the selected row index
            }

            # Call the handler if grid_manager is provided
            if grid_manager:
                grid_manager.update_deck_content(event, widget)
        else:
            print(f"Invalid row index {row_index}. Must be between 0 and {len(grid_df) - 1}.")
    else:
        print("No data in the grid to select.")

def trigger_sort_on_column(deck_content_grid, column_name, grid_manager=None):
    """
    Programmatically triggers a sort on a specific column in the qgrid widget.
    """
    # Get the grid DataFrame
    
    grid_df = deck_content_grid.get_changed_df()
    
    if grid_df is not None and column_name in grid_df.columns:
                
        # Trigger the sort_changed event manually
        event = {
            'new': {
                'column': column_name,
                'ascending': True  # Adjust this based on desired sort order
            }
        }
        
        # Call the grid manager's sort handler
        if grid_manager:
            grid_manager.handle_sort_changed(event, deck_content_grid)
    else:
        print(f"Column '{column_name}' not found in the grid.")

# Initialize the interface
pgui.setup_interface()
pgui.db_list.value = 'strudel'

test_update_filter_grid(pgui.grid_manager)

add_row_in_qgrid(pgui.grid_manager, 1)

# Assuming you want to select the first row in the filter grid and trigger an update
select_row_in_qgrid(pgui.grid_manager.qm, 1, 0, pgui.grid_manager)
pgui.grid_manager.open_deck('filtered_grid_1', None)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Sample DataFrame for demonstration purposes
df = pd.DataFrame({
    col: [i % 5 for i in range(100)] for col in [
        'Name', 'registeredDate', 'UpdatedAt', 'xp', 'elo', 'level', 'pExpiry', 'cardSetNo', 'faction', 'forgebornId',
        'Deck A', 'Deck B', 'faction', 'crossFaction', 'forgebornId',
        'A1', 'H1', 'A2', 'H2', 'A3', 'H3', 'Creatures', 'Spells', 'Exalts',
        'Beast Synergy', 'Mage Synergy', 'Robot Synergy', 'Scientist Synergy', 'Spirit Synergy', 'Warrior Synergy',
        'Zombie Synergy', 'Activate Combo', 'Upgrade Synergy', 'Face Burn Combo', 'Removal Synergy', 'Silence Combo',
        'Annoying', 'Fun', 'Cool', 'Spicy', 'Last Winter'
    ]
})

# Groups of columns organized by categories
groups = {
    'Deck Data': ['Name', 'registeredDate', 'UpdatedAt', 'xp', 'elo', 'level', 'pExpiry', 'cardSetNo', 'faction', 'forgebornId'],
    'Fusion Data': ['Name', 'Deck A', 'Deck B', 'faction', 'crossFaction', 'forgebornId'],
    'Statistical Data': ['A1', 'H1', 'A2', 'H2', 'A3', 'H3', 'Creatures', 'Spells', 'Exalts'],
    'Synergies': [col for col in df.columns if 'Synergy' in col],
    'Combos': [col for col in df.columns if 'Combo' in col],
    'Action and Ability Attributes': ['Activate', 'Ready', 'Free', 'Upgrade', 'Face Burn', 'Removal', 'Silence', 'Deploy', 'Slay', 'Reanimate'] + 
                                      [col for col in df.columns if any(term in col for term in ['Synergy', 'Combo'])],
    'Miscellaneous Attributes': ['Annoying', 'Fun', 'Cool', 'Spicy', 'Last Winter']
}

# Widget to select groups
group_selector = widgets.SelectMultiple(
    options=groups.keys(),
    description="",
    disabled=False
)
group_selector_label = widgets.Label("Select Groups:")

# Widget to display columns based on group selection
column_selector = widgets.SelectMultiple(
    options=[],
    description="",
    disabled=False
)
column_selector_label = widgets.Label("Select Columns:")

def update_column_selector(*args):
    selected_groups = group_selector.value
    selected_columns = [col for group in selected_groups for col in groups[group]]
    column_selector.options = selected_columns

group_selector.observe(update_column_selector, 'value')

# Toggle buttons for show/hide
toggle_buttons = widgets.ToggleButtons(
    options=['Show Selected', 'Hide Selected'],
    description='',
    disabled=False
)
toggle_buttons_label = widgets.Label("Set Visibility:")

# Output widget to display changes and messages
output = widgets.Output()

def toggle_visibility(change):
    columns_to_toggle = column_selector.value
    action = change.new

    with output:
        clear_output(wait=True)
        if action == 'Show Selected':
            print(f"Showing columns: {', '.join(columns_to_toggle)}")
        elif action == 'Hide Selected':
            print(f"Hiding columns: {', '.join(columns_to_toggle)}")

toggle_buttons.observe(toggle_visibility, 'value')

# Button to lock/unlock configurations
lock_button = widgets.Button(
    description='Lock',
    disabled=False,
    button_style='',
    tooltip='Click to lock/unlock the current configuration',
    icon='lock'
)
lock_button_label = widgets.Label("Lock/Unlock Configurations:")

def lock_configuration(b):
    if lock_button.description == 'Lock':
        lock_button.description = 'Unlock'
        lock_button.icon = 'unlock'
        group_selector.disabled = True
        column_selector.disabled = True
        toggle_buttons.disabled = True
    else:
        lock_button.description = 'Lock'
        lock_button.icon = 'lock'
        group_selector.disabled = False
        column_selector.disabled = False
        toggle_buttons.disabled = False

    with output:
        clear_output()
        print(f"Configuration {'locked' if lock_button.description == 'Unlock' else 'unlocked'}.")

lock_button.on_click(lock_configuration)

# Arranging widgets with labels above
group_selector_container = widgets.VBox([group_selector_label, group_selector])
column_selector_container = widgets.VBox([column_selector_label, column_selector])
toggle_buttons_container = widgets.VBox([toggle_buttons_label, toggle_buttons])
lock_button_container = widgets.VBox([lock_button_label, lock_button])

# Display the widgets in a horizontal layout
display(widgets.HBox([group_selector_container, column_selector_container, toggle_buttons_container, lock_button_container]), output)

In [ ]:
import ipywidgets as widgets
import pandas as pd
import qgrid

# Sample DataFrame for the qgrid
sample_data = pd.DataFrame({
    'Column A': ['Row 1', 'Row 2', 'Row 3'],
    'Column B': [10, 20, 30],
    'Column C': ['Alpha', 'Beta', 'Gamma']
})

# Create a qgrid widget with the sample data
qgrid_widget = qgrid.show_grid(sample_data, show_toolbar=True, grid_options={'forceFitColumns': True})

# Define a GridSpecLayout with 6 rows and 1 column
num_grids = 6
grid_layout = widgets.GridspecLayout(num_grids, 1, height="auto")

# Add the qgrid to the first row
grid_layout[0, 0] = qgrid_widget

# Fill the remaining rows with empty VBox widgets
for i in range(1, num_grids):
    grid_layout[i, 0] = widgets.Label("No active filters.")

# Display the layout
grid_layout


In [ ]:
import pygui as pgui
pgui.setup_interface()
pgui.db_list.value = 'tmind'

pgui.selected_items_label.value = 'The Smelly Fooling Sentinels'
pgui.display_graph()